**Plan:**


**1. Concurrency and parallelism in Python**

**2. Python's GIL (Global Interpreter Lock)**

**3. Asyncio and asynchronous programming**


# **1. Concurrency and parallelism in Python**


Concurrency and parallelism are concepts related to the execution of multiple tasks or processes in Python:

1. **Concurrency:**
   Concurrency refers to the ability of a Python program to handle multiple tasks or processes simultaneously. In concurrent programming, tasks may not run simultaneously but are interleaved or overlapped in execution. Concurrency is typically achieved through techniques such as multitasking, multithreading, or asynchronous programming. Python's `asyncio` module and `threading` module are commonly used for implementing concurrency.

2. **Parallelism:**
   Parallelism, on the other hand, refers to the ability of a Python program to execute multiple tasks or processes simultaneously by utilizing multiple CPU cores or processors. In parallel programming, tasks are truly executed concurrently, with each task running simultaneously on a separate CPU core or processor. Parallelism is often achieved through techniques such as multiprocessing or parallel computing libraries like `multiprocessing` or `joblib` in Python.

In summary, concurrency focuses on managing and coordinating multiple tasks efficiently, while parallelism focuses on executing multiple tasks simultaneously to achieve improved performance, especially on multi-core systems. Python provides various libraries and modules to support both concurrency and parallelism, allowing developers to write efficient and scalable concurrent and parallel programs.

**Concurrency Example using asyncio**

In [ ]:
# The code will not work on colab
import asyncio

# Define a coroutine function
async def greet(name):
    print(f"Hello, {name}!")
    # NB: if a function uses sleep, it allows other tasks to run concurrently while it's running
    await asyncio.sleep(1)
    print(f"Goodbye, {name}!")

# Run coroutines concurrently
async def main():
    await asyncio.gather(
        greet("Alice"),
        greet("Bob"),
        greet("Charlie")
    )

# Call asyncio.run() to start the event loop
asyncio.run(main())


In [3]:
# Move the code below on this file
! touch concurrency.py

In [4]:
! python3 concurrency.py

Hello, Alice!
Hello, Bob!
Hello, Charlie!
Goodbye, Alice!
Goodbye, Bob!
Goodbye, Charlie!



**Difference between function with async and without async:**
- **Without `async`:** In the synchronous version, `factorial()` is a regular function that calculates the factorial of a number synchronously using a loop. While the calculation is in progress, the program is blocked, and no other tasks can be executed.
- **With `async`:** In the asynchronous version, `factorial_async()` is an asynchronous function defined with `async`. Although in this simple example it doesn't involve I/O, the function is designed to be non-blocking and allows other tasks to run concurrently while it's running. The `await asyncio.sleep(0.1)` line simulates an I/O-bound operation (e.g., network request, file I/O) to demonstrate the potential for concurrency.

In summary, the asynchronous version allows for non-blocking execution, enabling other tasks to run concurrently while the factorial calculation is in progress. This can be particularly beneficial for I/O-bound operations, where the program would otherwise be waiting for external resources.

In [ ]:
# Function to calculate factorial synchronously (blocking)
def factorial(n):
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result

# Example usage
num = 5
fact = factorial(num)
print(f"The factorial of {num} is {fact}")


In [ ]:
import asyncio

# Asynchronous function to calculate factorial asynchronously (non-blocking)
async def factorial_async(n):
    result = 1
    for i in range(1, n + 1):
        result *= i
        # Simulate I/O-bound operation
        await asyncio.sleep(0.1)
    return result

# Example usage
async def main():
    num = 5
    fact = await factorial_async(num)
    print(f"The factorial of {num} is {fact}")

asyncio.run(main())

**Function without asynch and implement sleep I/O**
No, adding `time.sleep()` in a synchronous function like `factorial()` will not make it non-blocking. `time.sleep()` is a blocking operation that pauses the execution of the current thread for a specified number of seconds, causing the program to wait until the sleep duration elapses before continuing.

In this case, each iteration of the loop in `factorial()` will pause the execution of the current thread for 0.1 seconds, blocking the program's execution. Other tasks or operations in the program will be delayed until the sleep operation completes.

Therefore, even though `time.sleep()` introduces a delay, it does not enable other tasks to run concurrently. To achieve non-blocking behavior and enable concurrency, you would need to use asynchronous programming with `asyncio` and `await` syntax, as shown in the previous example with `async def` and `await asyncio.sleep()`.

In [ ]:
import time

# Synchronous function to calculate factorial (blocking)
def factorial(n):
    result = 1
    for i in range(1, n + 1):
        result *= i
        # Simulate blocking I/O operation
        # Even the function use sleep, It blocks all other functions
        time.sleep(0.1)  # Blocking sleep
    return result


**Resume**

Functions that do not use `async` (synchronous functions) are blocking even if they perform input/output operations or use `wait` or `sleep`; they do not yield control to other processes. On the other hand, functions that use `async` yield control if they perform input/output operations, `wait`, or `sleep`.

**Parallelism Example using multiprocessing:**

In [6]:
# Parallel sum
# Queue is used for processes communication
from multiprocessing import Process, Queue

# Function to perform addition on a sublist
def sublist_add(sublist, result_queue):
    result = sum(sublist)
    result_queue.put(result)

if __name__ == "__main__":
    # Input data
    data = [list(range(1, 101)), list(range(101, 201)), list(range(201, 301)), list(range(301, 401))]

    # Create a result queue to store the results
    result_queue = Queue()

    # Create processes
    processes = []
    for sublist in data:
        p = Process(target=sublist_add, args=(sublist, result_queue))
        processes.append(p)
        p.start()

    # Wait for all processes to finish
    for p in processes:
        p.join()

    # Retrieve results from the queue
    results = []
    while not result_queue.empty():
        results.append(result_queue.get())

    # Print results
    print("Results:", results)


Results: [5050, 15050, 25050, 35050]


In [7]:
# Parallel multiplication
from multiprocessing import Process, Queue

# Function to perform multiplication on a sublist
def sublist_multiply(sublist, result_queue):
    result = 1
    for num in sublist:
        result *= num
    result_queue.put(result)

if __name__ == "__main__":
    # Input data
    data = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]

    # Create a result queue to store the results
    result_queue = Queue()

    # Create processes
    processes = []
    for sublist in data:
        p = Process(target=sublist_multiply, args=(sublist, result_queue))
        processes.append(p)
        p.start()

    # Wait for all processes to finish
    for p in processes:
        p.join()

    # Retrieve results from the queue
    results = []
    while not result_queue.empty():
        results.append(result_queue.get())

    # Print results
    print("Results:", results)


Results: [24, 1680, 11880, 43680]


# **2. Python's GIL (Global Interpreter Lock)**

The Global Interpreter Lock (GIL) in Python is a mutex (mutual exclusion) that protects access to Python objects, preventing multiple native threads from executing Python bytecodes concurrently in the same process. This means that only one thread can execute Python bytecode at a time, regardless of the number of CPU cores available.

The GIL is a design feature of the Python interpreter (CPython) and is intended to simplify memory management and provide thread safety by ensuring that only one thread executes Python bytecode at a time. However, this can lead to performance limitations, particularly in CPU-bound multithreaded applications, as it prevents true parallelism and limits the utilization of multiple CPU cores.

It's important to note that the GIL only affects CPython, the reference implementation of Python. Alternative implementations such as Jython and IronPython do not have a GIL and can execute multiple threads concurrently. Additionally, asynchronous programming with `asyncio` can be used to achieve concurrency without relying on threads and bypassing the GIL.

**Example 1 with Threads**

- We define a function `cpu_bound_calculation()` to perform a CPU-bound calculation. Each thread is assigned a unique identifier.
- We define a function `write_to_file()` to write the result to a file. This function is protected by a lock (`file_lock`) to ensure that only one thread can write to the file at a time.
- We define a function `calculate_and_write()` to perform the calculation and write the result to the file. This function is called by each thread.
- In the `run_calculations()` function, we create four threads, each calling `calculate_and_write()` to perform the calculation and write the result to the file.
- We join all the threads to wait for them to finish execution.

This example demonstrates how to safely write to a file from multiple threads in Python, ensuring that only one thread writes to the file at a time using a lock.

In [8]:
import threading

# Function to perform a CPU-bound calculation
def cpu_bound_calculation(thread_id):
    result = 0
    for _ in range(10**7):
        result += 1
    return f"Thread {thread_id}: {result}\n"

# Function to write result to file
def write_to_file(file, result):
    # Lock variable access
    with file_lock:
        file.write(result)

# Function to perform calculation and write result
def calculate_and_write(thread_id, file):
    result = cpu_bound_calculation(thread_id)
    write_to_file(file, result)

# Create a lock for file access
file_lock = threading.Lock()

# Function to run CPU-bound calculations and write results to file
def run_calculations():
    with open("results.txt", "a") as file:
        threads = []
        for i in range(4):  # Create 4 threads
            thread = threading.Thread(target=calculate_and_write, args=(i, file))
            thread.start()
            threads.append(thread)
        for thread in threads:
            thread.join()

# Run CPU-bound calculations and write results to file
run_calculations()



**Example 2 with Threads**
- We have a shared variable `shared_variable` that is initially set to 0.
- We use a lock (`lock`) to protect access to the shared variable.
- We define three functions: `increment_shared_variable()`, `decrement_shared_variable()`, and `print_shared_variable()`, each of which accesses the shared variable within a `with lock` block.
- The `perform_operations()` function is designed to increment and decrement the shared variable multiple times in a loop while printing its value.
- We create four threads to call `perform_operations()` concurrently.
- Each thread increments and decrements the shared variable within the `with lock` block to ensure thread safety.
- After all threads finish execution, we print the final value of the shared variable.

This example demonstrates how to safely manipulate a shared variable from multiple threads using a lock in Python.

In [ ]:
import threading

# Shared variable
shared_variable = 0

# Lock for shared_variable access
lock = threading.Lock()

# Function to increment the shared variable
def increment_shared_variable():
    global shared_variable
    with lock:
        shared_variable += 1

# Function to decrement the shared variable
def decrement_shared_variable():
    global shared_variable
    with lock:
        shared_variable -= 1

# Function to print the shared variable
def print_shared_variable():
    with lock:
        print("Shared Variable:", shared_variable)

# Function to perform operations on the shared variable
def perform_operations():
    for _ in range(100000):
        increment_shared_variable()
        decrement_shared_variable()
        print_shared_variable()

# Create threads to perform operations
threads = []
for _ in range(4):
    thread = threading.Thread(target=perform_operations)
    thread.start()
    threads.append(thread)

# Wait for all threads to finish
for thread in threads:
    thread.join()

# Print the final value of the shared variable
print("Final Shared Variable:", shared_variable)


In Python, threads and processes are both mechanisms for parallel execution, but they differ in several key aspects:

1. **Memory**:
   - **Threads**: Threads within the same process share the same memory space, including global variables, heap memory, and other resources. This means that data can be easily shared between threads.
   - **Processes**: Processes have separate memory spaces. Each process has its own address space, so data is not shared between processes by default. Inter-process communication mechanisms like pipes, queues, or shared memory need to be used to share data between processes.

2. **Execution**:
   - **Threads**: Threads are lighter-weight than processes and share the same CPU core. They are managed by the operating system's thread scheduler, which switches between threads to give the appearance of concurrent execution.
   - **Processes**: Processes are heavier-weight than threads because they have their own memory space and resources. Each process has its own independent execution context and is managed by the operating system's process scheduler.

3. **Concurrency vs. Parallelism**:
   - **Concurrency**: Threads enable concurrent execution, allowing multiple tasks to be executed in an interleaved manner. In Python, due to the Global Interpreter Lock (GIL) in CPython, true parallelism is not achieved with threads for CPU-bound tasks (On thread is executed at a given time).
   - **Parallelism**: Processes enable true parallel execution, where multiple tasks are executed simultaneously on multiple CPU cores. Parallelism can be achieved by using multiple processes, which can run independently of each other.

4. **Communication**:
   - **Threads**: Communication between threads is straightforward since they share memory. However, synchronization mechanisms like locks, semaphores, and condition variables are needed to ensure thread safety.
   - **Processes**: Communication between processes is more complex because they don't share memory. Inter-process communication (IPC) mechanisms such as pipes, queues, sockets, and shared memory need to be used for communication between processes.

**In summary, threads are suitable for I/O-bound tasks and situations where shared memory is required, while processes are more suitable for CPU-bound tasks and situations where isolation and true parallelism are needed.**

**French: En résumé, les threads sont adaptés aux tâches liées à l'entrée/sortie et aux situations où la mémoire partagée est requise (si un thread fais un I/O, le GIL est relâché, et les autres threads peuvent s'exécuter), tandis que les processus sont plus adaptés aux tâches liées au processeur et aux situations où l'isolation et la véritable parallélisme sont nécessaires.**